In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("train.csv")

In [3]:
#RANDOM_STATE to reproduce the same outcomes 
RANDOM_STATE = 15

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
one_hot_needed = [
    'Pclass',
    'Sex',
    'Embarked'
]
df = pd.get_dummies(data = data,
                         prefix = one_hot_needed,
                         columns = one_hot_needed)

In [6]:
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0,1,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,1,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,1,0,0,0,1
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,0,1,0,1,0,0,1


In [7]:
#dropping 'PassengerId','Name','Ticket','Cabin' and target -> "Survived"
features_df = df.drop(['PassengerId','Name','Ticket','Cabin',"Survived"], axis=1)

features = features_df.columns.tolist()

In [8]:
df.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,1,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0,1,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,1,0,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,1,0,0,0,1
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,0,1,0,1,0,0,1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df.fillna(0,inplace = True)
x_train, x_cv, y_train, y_cv = train_test_split(df[features], df['Survived'], train_size = 0.8, random_state = RANDOM_STATE)
print(f'Training samples: {x_train.shape}')
print(f'Cross-validation samples: {x_cv.shape}')


Training samples: (712, 12)
Cross-validation samples: (179, 12)


In [10]:
#dTree classifier
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(min_samples_split = 20,max_depth = 4,
                                             random_state = RANDOM_STATE).fit(x_train[features],y_train)


In [11]:
#hyperparameter tuning

from sklearn.model_selection import GridSearchCV

print("Accuracy Scores (DecisionTreeClassifier):\n")
print(f"Train: {accuracy_score(decision_tree_model.predict(x_train),y_train)}")
print(f"Cross-validation: {accuracy_score(decision_tree_model.predict(x_cv),y_cv)}")

param_grid = {
    'min_samples_split': [2**i for i in range(1,12)],
    'max_depth': [2**i for i in range(10)]
}

model = DecisionTreeClassifier(random_state = RANDOM_STATE)

#cv = 5 -> 5-fold cross-validation; each fold acts as a cv once
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

grid_search.fit(x_train, y_train)

print("\nGridSearchCV (DecisionTreeClassifier) \n")
print("Best parameters found: ", grid_search.best_params_)
print("Best estimator found: ", grid_search.best_estimator_)
print("Best score found: ", grid_search.best_score_)

Accuracy Scores (DecisionTreeClassifier):

Train: 0.8216292134831461
Cross-validation: 0.7932960893854749

GridSearchCV (DecisionTreeClassifier) 

Best parameters found:  {'max_depth': 8, 'min_samples_split': 16}
Best estimator found:  DecisionTreeClassifier(max_depth=8, min_samples_split=16, random_state=15)
Best score found:  0.8089530188121736


In [12]:
#dtree 1 

from sklearn.model_selection import cross_val_score
classifier = DecisionTreeClassifier(max_depth=8, min_samples_split = 16)
scores = cross_val_score(classifier, df[features].to_numpy(), df["Survived"].to_numpy(), cv=8, scoring='accuracy')
mean_score = scores.mean()

print(f"Mean accuracy: {mean_score}")

Mean accuracy: 0.8071408462033463


In [13]:
from xgboost import XGBClassifier

XGB_model = XGBClassifier(n_estimators=10000, learning_rate=0.02, verbosity=1,
                          random_state=RANDOM_STATE, eval_metric='logloss')
XGB_model.set_params(early_stopping_rounds=1000)
XGB_model.fit(x_train[features],y_train, eval_set=[(x_cv[features], y_cv)])


[0]	validation_0-logloss:0.68339
[1]	validation_0-logloss:0.67451
[2]	validation_0-logloss:0.66567
[3]	validation_0-logloss:0.65744
[4]	validation_0-logloss:0.64924
[5]	validation_0-logloss:0.64161
[6]	validation_0-logloss:0.63423
[7]	validation_0-logloss:0.62717
[8]	validation_0-logloss:0.61998
[9]	validation_0-logloss:0.61343
[10]	validation_0-logloss:0.60705
[11]	validation_0-logloss:0.60099
[12]	validation_0-logloss:0.59471
[13]	validation_0-logloss:0.58937
[14]	validation_0-logloss:0.58398
[15]	validation_0-logloss:0.57870
[16]	validation_0-logloss:0.57376
[17]	validation_0-logloss:0.56898
[18]	validation_0-logloss:0.56445
[19]	validation_0-logloss:0.55976
[20]	validation_0-logloss:0.55552
[21]	validation_0-logloss:0.55126
[22]	validation_0-logloss:0.54723
[23]	validation_0-logloss:0.54325
[24]	validation_0-logloss:0.53961
[25]	validation_0-logloss:0.53594
[26]	validation_0-logloss:0.53258
[27]	validation_0-logloss:0.52868
[28]	validation_0-logloss:0.52543
[29]	validation_0-loglos

[238]	validation_0-logloss:0.42228
[239]	validation_0-logloss:0.42244
[240]	validation_0-logloss:0.42212
[241]	validation_0-logloss:0.42221
[242]	validation_0-logloss:0.42221
[243]	validation_0-logloss:0.42237
[244]	validation_0-logloss:0.42221
[245]	validation_0-logloss:0.42202
[246]	validation_0-logloss:0.42219
[247]	validation_0-logloss:0.42194
[248]	validation_0-logloss:0.42197
[249]	validation_0-logloss:0.42213
[250]	validation_0-logloss:0.42201
[251]	validation_0-logloss:0.42187
[252]	validation_0-logloss:0.42187
[253]	validation_0-logloss:0.42198
[254]	validation_0-logloss:0.42217
[255]	validation_0-logloss:0.42204
[256]	validation_0-logloss:0.42219
[257]	validation_0-logloss:0.42207
[258]	validation_0-logloss:0.42197
[259]	validation_0-logloss:0.42202
[260]	validation_0-logloss:0.42193
[261]	validation_0-logloss:0.42181
[262]	validation_0-logloss:0.42195
[263]	validation_0-logloss:0.42199
[264]	validation_0-logloss:0.42190
[265]	validation_0-logloss:0.42211
[266]	validation_0-l

[473]	validation_0-logloss:0.43390
[474]	validation_0-logloss:0.43382
[475]	validation_0-logloss:0.43390
[476]	validation_0-logloss:0.43406
[477]	validation_0-logloss:0.43410
[478]	validation_0-logloss:0.43411
[479]	validation_0-logloss:0.43421
[480]	validation_0-logloss:0.43428
[481]	validation_0-logloss:0.43432
[482]	validation_0-logloss:0.43436
[483]	validation_0-logloss:0.43443
[484]	validation_0-logloss:0.43444
[485]	validation_0-logloss:0.43449
[486]	validation_0-logloss:0.43450
[487]	validation_0-logloss:0.43456
[488]	validation_0-logloss:0.43458
[489]	validation_0-logloss:0.43466
[490]	validation_0-logloss:0.43481
[491]	validation_0-logloss:0.43498
[492]	validation_0-logloss:0.43509
[493]	validation_0-logloss:0.43526
[494]	validation_0-logloss:0.43534
[495]	validation_0-logloss:0.43554
[496]	validation_0-logloss:0.43558
[497]	validation_0-logloss:0.43559
[498]	validation_0-logloss:0.43564
[499]	validation_0-logloss:0.43572
[500]	validation_0-logloss:0.43583
[501]	validation_0-l

[708]	validation_0-logloss:0.45434
[709]	validation_0-logloss:0.45440
[710]	validation_0-logloss:0.45478
[711]	validation_0-logloss:0.45484
[712]	validation_0-logloss:0.45494
[713]	validation_0-logloss:0.45516
[714]	validation_0-logloss:0.45510
[715]	validation_0-logloss:0.45509
[716]	validation_0-logloss:0.45507
[717]	validation_0-logloss:0.45523
[718]	validation_0-logloss:0.45527
[719]	validation_0-logloss:0.45560
[720]	validation_0-logloss:0.45566
[721]	validation_0-logloss:0.45578
[722]	validation_0-logloss:0.45561
[723]	validation_0-logloss:0.45574
[724]	validation_0-logloss:0.45610
[725]	validation_0-logloss:0.45615
[726]	validation_0-logloss:0.45622
[727]	validation_0-logloss:0.45664
[728]	validation_0-logloss:0.45680
[729]	validation_0-logloss:0.45693
[730]	validation_0-logloss:0.45734
[731]	validation_0-logloss:0.45725
[732]	validation_0-logloss:0.45736
[733]	validation_0-logloss:0.45748
[734]	validation_0-logloss:0.45753
[735]	validation_0-logloss:0.45757
[736]	validation_0-l

[943]	validation_0-logloss:0.47110
[944]	validation_0-logloss:0.47112
[945]	validation_0-logloss:0.47140
[946]	validation_0-logloss:0.47139
[947]	validation_0-logloss:0.47144
[948]	validation_0-logloss:0.47163
[949]	validation_0-logloss:0.47166
[950]	validation_0-logloss:0.47185
[951]	validation_0-logloss:0.47193
[952]	validation_0-logloss:0.47203
[953]	validation_0-logloss:0.47206
[954]	validation_0-logloss:0.47215
[955]	validation_0-logloss:0.47240
[956]	validation_0-logloss:0.47261
[957]	validation_0-logloss:0.47267
[958]	validation_0-logloss:0.47269
[959]	validation_0-logloss:0.47272
[960]	validation_0-logloss:0.47276
[961]	validation_0-logloss:0.47275
[962]	validation_0-logloss:0.47277
[963]	validation_0-logloss:0.47282
[964]	validation_0-logloss:0.47293
[965]	validation_0-logloss:0.47308
[966]	validation_0-logloss:0.47307
[967]	validation_0-logloss:0.47320
[968]	validation_0-logloss:0.47326
[969]	validation_0-logloss:0.47328
[970]	validation_0-logloss:0.47339
[971]	validation_0-l

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=1000,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=10000, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=15, ...)

In [14]:
try:
    scores = cross_val_score(XGB_model, df[features].to_numpy(), df["Survived"].to_numpy(), cv=8, scoring='accuracy', error_score='raise')
    mean_score = scores.mean()
    print(f"Mean accuracy: {mean_score}")
except Exception as e:
    print(f"Error during model fitting: {e}")
    
    
print("XGB scores:\n")
print("Best Iteration: ",XGB_model.best_iteration)
print(f"Train Accuracy: {accuracy_score(XGB_model.predict(x_train),y_train)}")
print(f"Cross-validation Accuracy: {accuracy_score(XGB_model.predict(x_cv),y_cv)}")

Error during model fitting: Must have at least 1 validation dataset for early stopping.
XGB scores:

Best Iteration:  155
Train Accuracy: 0.8876404494382022
Cross-validation Accuracy: 0.8491620111731844


In [15]:
'''
#experimenting 

from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
import numpy as np

import warnings
warnings.filterwarnings('ignore')

class CustomXGBClassifier(XGBClassifier):
    def fit(self, X, y, **fit_params):
        # Split the data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        super().fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, **fit_params)

# Create the custom model
XGB_model = CustomXGBClassifier(n_estimators=100, learning_rate=0.1, verbosity=1,
                                random_state=RANDOM_STATE, eval_metric='logloss')

# Perform cross-validation
scores = cross_validate(XGB_model, df[features].to_numpy(), df["Survived"].to_numpy(), cv=8, 
                        scoring='accuracy', return_train_score=True)

# Calculate the mean score
mean_train_score = scores['train_score'].mean()
mean_test_score = scores['test_score'].mean()

print(f"Mean train accuracy: {mean_train_score}")
print(f"Mean validation accuracy: {mean_test_score}")
'''

'\nfrom sklearn.model_selection import cross_validate\nfrom sklearn.metrics import accuracy_score\nimport numpy as np\n\nimport warnings\nwarnings.filterwarnings(\'ignore\')\n\nclass CustomXGBClassifier(XGBClassifier):\n    def fit(self, X, y, **fit_params):\n        # Split the data into training and validation sets\n        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)\n        super().fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, **fit_params)\n\n# Create the custom model\nXGB_model = CustomXGBClassifier(n_estimators=100, learning_rate=0.1, verbosity=1,\n                                random_state=RANDOM_STATE, eval_metric=\'logloss\')\n\n# Perform cross-validation\nscores = cross_validate(XGB_model, df[features].to_numpy(), df["Survived"].to_numpy(), cv=8, \n                        scoring=\'accuracy\', return_train_score=True)\n\n# Calculate the mean score\nmean_train_score = scores[\'train_score\'].mean()\nme

In [16]:
test = pd.read_csv("test.csv")
test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
#one_hot_encoding test
df_test = pd.get_dummies(data = test,
                         prefix = one_hot_needed,
                         columns = one_hot_needed)


df_test[features].head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
1,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
2,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
3,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
4,22.0,1,1,12.2875,0,0,1,1,0,0,0,1


In [18]:
predictions = XGB_model.predict(df_test[features])
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submissionXGB.csv', index=False)
print("Saved!")


Saved!


In [19]:
XGB_model2 = XGBClassifier(n_estimators=10000, learning_rate=0.02, verbosity=1,
                          random_state=RANDOM_STATE, eval_metric='logloss')
XGB_model2.fit(df[features].to_numpy(), df["Survived"].to_numpy())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.02, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=10000, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=15, ...)

In [20]:
predictions = XGB_model2.predict(df_test[features])
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submissionXGB2.csv', index=False)
print("Saved!")

Saved!


In [21]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(min_samples_split = 16,max_depth = 10,
                                             random_state = RANDOM_STATE,n_jobs = 10, n_estimators = 1000).fit(x_train,y_train)
scores = cross_val_score(random_forest_model, df[features].to_numpy(), df["Survived"].to_numpy(), cv=8, scoring='accuracy')
mean_score = scores.mean()

df_test[features].fillna(0)
predictions = random_forest_model.predict(df_test[features].fillna(0))
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submissionRF.csv', index=False)
print("Success!")

print(f"Mean accuracy: {mean_score}")

Success!
Mean accuracy: 0.828356258043758


In [23]:
num_classifiers = 12
classifiers = [RandomForestClassifier(min_samples_split=16, max_depth=10,
                                      random_state=RANDOM_STATE+i, n_jobs=10, n_estimators=1000) 
               for i in range(num_classifiers)]

# training each of the classifiers
for i, clf in enumerate(classifiers):
    clf.fit(x_train, y_train)

# making predictions with each classifier
predictions = [clf.predict(df_test[features].fillna(0)) for clf in classifiers]

# applying the majority voting rule
predictions_sum = np.sum(predictions, axis=0)
final_predictions = np.where(predictions_sum > 8, 1, 0)

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': final_predictions})
output.to_csv('submissionRF12.csv', index=False)

print("Success!")


Success!


In [24]:
num_classifiers_rf = 25
num_classifiers_xgb = 15

classifiers_rf = [RandomForestClassifier(min_samples_split=16, max_depth=10,
                                         random_state=RANDOM_STATE+i, n_jobs=10, n_estimators=1000) 
                  for i in range(num_classifiers_rf)]

classifiers_xgb = [XGBClassifier(n_estimators=10000, learning_rate=0.02, verbosity=1,
                                 random_state=RANDOM_STATE+i, eval_metric='logloss', early_stopping_rounds=1000) 
                   for i in range(num_classifiers_xgb)]

#training each of the RandomForest classifiers
for i, clf in enumerate(classifiers_rf):
    clf.fit(x_train[features], y_train)

# training each of the XGBoost classifiers with evaluation set
for i, clf in enumerate(classifiers_xgb):
    clf.fit(x_train[features], y_train, eval_set=[(x_cv[features], y_cv)])

# making predictions with each classifier
predictions = [clf.predict(df_test[features].fillna(0)) for clf in classifiers_rf + classifiers_xgb]

# applying the majority voting rule
predictions_sum = np.sum(predictions, axis=0)
final_predictions = np.where(predictions_sum > 34, 1, 0)  # assuming that the majority is more than half

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': final_predictions})
output.to_csv('submissionXGBrf40.csv', index=False)

print("Success!")

[0]	validation_0-logloss:0.68339
[1]	validation_0-logloss:0.67451
[2]	validation_0-logloss:0.66567
[3]	validation_0-logloss:0.65744
[4]	validation_0-logloss:0.64924
[5]	validation_0-logloss:0.64161
[6]	validation_0-logloss:0.63423
[7]	validation_0-logloss:0.62717
[8]	validation_0-logloss:0.61998
[9]	validation_0-logloss:0.61343
[10]	validation_0-logloss:0.60705
[11]	validation_0-logloss:0.60099
[12]	validation_0-logloss:0.59471
[13]	validation_0-logloss:0.58937
[14]	validation_0-logloss:0.58398
[15]	validation_0-logloss:0.57870
[16]	validation_0-logloss:0.57376
[17]	validation_0-logloss:0.56898
[18]	validation_0-logloss:0.56445
[19]	validation_0-logloss:0.55976
[20]	validation_0-logloss:0.55552
[21]	validation_0-logloss:0.55126
[22]	validation_0-logloss:0.54723
[23]	validation_0-logloss:0.54325
[24]	validation_0-logloss:0.53961
[25]	validation_0-logloss:0.53594
[26]	validation_0-logloss:0.53258
[27]	validation_0-logloss:0.52868
[28]	validation_0-logloss:0.52543
[29]	validation_0-loglos

[238]	validation_0-logloss:0.42228
[239]	validation_0-logloss:0.42244
[240]	validation_0-logloss:0.42212
[241]	validation_0-logloss:0.42221
[242]	validation_0-logloss:0.42221
[243]	validation_0-logloss:0.42237
[244]	validation_0-logloss:0.42221
[245]	validation_0-logloss:0.42202
[246]	validation_0-logloss:0.42219
[247]	validation_0-logloss:0.42194
[248]	validation_0-logloss:0.42197
[249]	validation_0-logloss:0.42213
[250]	validation_0-logloss:0.42201
[251]	validation_0-logloss:0.42187
[252]	validation_0-logloss:0.42187
[253]	validation_0-logloss:0.42198
[254]	validation_0-logloss:0.42217
[255]	validation_0-logloss:0.42204
[256]	validation_0-logloss:0.42219
[257]	validation_0-logloss:0.42207
[258]	validation_0-logloss:0.42197
[259]	validation_0-logloss:0.42202
[260]	validation_0-logloss:0.42193
[261]	validation_0-logloss:0.42181
[262]	validation_0-logloss:0.42195
[263]	validation_0-logloss:0.42199
[264]	validation_0-logloss:0.42190
[265]	validation_0-logloss:0.42211
[266]	validation_0-l

[473]	validation_0-logloss:0.43390
[474]	validation_0-logloss:0.43382
[475]	validation_0-logloss:0.43390
[476]	validation_0-logloss:0.43406
[477]	validation_0-logloss:0.43410
[478]	validation_0-logloss:0.43411
[479]	validation_0-logloss:0.43421
[480]	validation_0-logloss:0.43428
[481]	validation_0-logloss:0.43432
[482]	validation_0-logloss:0.43436
[483]	validation_0-logloss:0.43443
[484]	validation_0-logloss:0.43444
[485]	validation_0-logloss:0.43449
[486]	validation_0-logloss:0.43450
[487]	validation_0-logloss:0.43456
[488]	validation_0-logloss:0.43458
[489]	validation_0-logloss:0.43466
[490]	validation_0-logloss:0.43481
[491]	validation_0-logloss:0.43498
[492]	validation_0-logloss:0.43509
[493]	validation_0-logloss:0.43526
[494]	validation_0-logloss:0.43534
[495]	validation_0-logloss:0.43554
[496]	validation_0-logloss:0.43558
[497]	validation_0-logloss:0.43559
[498]	validation_0-logloss:0.43564
[499]	validation_0-logloss:0.43572
[500]	validation_0-logloss:0.43583
[501]	validation_0-l

[708]	validation_0-logloss:0.45434
[709]	validation_0-logloss:0.45440
[710]	validation_0-logloss:0.45478
[711]	validation_0-logloss:0.45484
[712]	validation_0-logloss:0.45494
[713]	validation_0-logloss:0.45516
[714]	validation_0-logloss:0.45510
[715]	validation_0-logloss:0.45509
[716]	validation_0-logloss:0.45507
[717]	validation_0-logloss:0.45523
[718]	validation_0-logloss:0.45527
[719]	validation_0-logloss:0.45560
[720]	validation_0-logloss:0.45566
[721]	validation_0-logloss:0.45578
[722]	validation_0-logloss:0.45561
[723]	validation_0-logloss:0.45574
[724]	validation_0-logloss:0.45610
[725]	validation_0-logloss:0.45615
[726]	validation_0-logloss:0.45622
[727]	validation_0-logloss:0.45664
[728]	validation_0-logloss:0.45680
[729]	validation_0-logloss:0.45693
[730]	validation_0-logloss:0.45734
[731]	validation_0-logloss:0.45725
[732]	validation_0-logloss:0.45736
[733]	validation_0-logloss:0.45748
[734]	validation_0-logloss:0.45753
[735]	validation_0-logloss:0.45757
[736]	validation_0-l

[943]	validation_0-logloss:0.47110
[944]	validation_0-logloss:0.47112
[945]	validation_0-logloss:0.47140
[946]	validation_0-logloss:0.47139
[947]	validation_0-logloss:0.47144
[948]	validation_0-logloss:0.47163
[949]	validation_0-logloss:0.47166
[950]	validation_0-logloss:0.47185
[951]	validation_0-logloss:0.47193
[952]	validation_0-logloss:0.47203
[953]	validation_0-logloss:0.47206
[954]	validation_0-logloss:0.47215
[955]	validation_0-logloss:0.47240
[956]	validation_0-logloss:0.47261
[957]	validation_0-logloss:0.47267
[958]	validation_0-logloss:0.47269
[959]	validation_0-logloss:0.47272
[960]	validation_0-logloss:0.47276
[961]	validation_0-logloss:0.47275
[962]	validation_0-logloss:0.47277
[963]	validation_0-logloss:0.47282
[964]	validation_0-logloss:0.47293
[965]	validation_0-logloss:0.47308
[966]	validation_0-logloss:0.47307
[967]	validation_0-logloss:0.47320
[968]	validation_0-logloss:0.47326
[969]	validation_0-logloss:0.47328
[970]	validation_0-logloss:0.47339
[971]	validation_0-l

...

[1]	validation_0-logloss:0.67451
[2]	validation_0-logloss:0.66567
[3]	validation_0-logloss:0.65744
[4]	validation_0-logloss:0.64924
[5]	validation_0-logloss:0.64161
[6]	validation_0-logloss:0.63423
[7]	validation_0-logloss:0.62717
[8]	validation_0-logloss:0.61998
[9]	validation_0-logloss:0.61343
[10]	validation_0-logloss:0.60705
[11]	validation_0-logloss:0.60099
[12]	validation_0-logloss:0.59471
[13]	validation_0-logloss:0.58937
[14]	validation_0-logloss:0.58398
[15]	validation_0-logloss:0.57870
[16]	validation_0-logloss:0.57376
[17]	validation_0-logloss:0.56898
[18]	validation_0-logloss:0.56445
[19]	validation_0-logloss:0.55976
[20]	validation_0-logloss:0.55552
[21]	validation_0-logloss:0.55126
[22]	validation_0-logloss:0.54723
[23]	validation_0-logloss:0.54325
[24]	validation_0-logloss:0.53961
[25]	validation_0-logloss:0.53594
[26]	validation_0-logloss:0.53258
[27]	validation_0-logloss:0.52868
[28]	validation_0-logloss:0.52543
[29]	validation_0-logloss:0.52244
[30]	validation_0-loglo

[239]	validation_0-logloss:0.42244
[240]	validation_0-logloss:0.42212
[241]	validation_0-logloss:0.42221
[242]	validation_0-logloss:0.42221
[243]	validation_0-logloss:0.42237
[244]	validation_0-logloss:0.42221
[245]	validation_0-logloss:0.42202
[246]	validation_0-logloss:0.42219
[247]	validation_0-logloss:0.42194
[248]	validation_0-logloss:0.42197
[249]	validation_0-logloss:0.42213
[250]	validation_0-logloss:0.42201
[251]	validation_0-logloss:0.42187
[252]	validation_0-logloss:0.42187
[253]	validation_0-logloss:0.42198
[254]	validation_0-logloss:0.42217
[255]	validation_0-logloss:0.42204
[256]	validation_0-logloss:0.42219
[257]	validation_0-logloss:0.42207
[258]	validation_0-logloss:0.42197
[259]	validation_0-logloss:0.42202
[260]	validation_0-logloss:0.42193
[261]	validation_0-logloss:0.42181
[262]	validation_0-logloss:0.42195
[263]	validation_0-logloss:0.42199
[264]	validation_0-logloss:0.42190
[265]	validation_0-logloss:0.42211
[266]	validation_0-logloss:0.42221
[267]	validation_0-l

[474]	validation_0-logloss:0.43382
[475]	validation_0-logloss:0.43390
[476]	validation_0-logloss:0.43406
[477]	validation_0-logloss:0.43410
[478]	validation_0-logloss:0.43411
[479]	validation_0-logloss:0.43421
[480]	validation_0-logloss:0.43428
[481]	validation_0-logloss:0.43432
[482]	validation_0-logloss:0.43436
[483]	validation_0-logloss:0.43443
[484]	validation_0-logloss:0.43444
[485]	validation_0-logloss:0.43449
[486]	validation_0-logloss:0.43450
[487]	validation_0-logloss:0.43456
[488]	validation_0-logloss:0.43458
[489]	validation_0-logloss:0.43466
[490]	validation_0-logloss:0.43481
[491]	validation_0-logloss:0.43498
[492]	validation_0-logloss:0.43509
[493]	validation_0-logloss:0.43526
[494]	validation_0-logloss:0.43534
[495]	validation_0-logloss:0.43554
[496]	validation_0-logloss:0.43558
[497]	validation_0-logloss:0.43559
[498]	validation_0-logloss:0.43564
[499]	validation_0-logloss:0.43572
[500]	validation_0-logloss:0.43583
[501]	validation_0-logloss:0.43597
[502]	validation_0-l

[709]	validation_0-logloss:0.45440
[710]	validation_0-logloss:0.45478
[711]	validation_0-logloss:0.45484
[712]	validation_0-logloss:0.45494
[713]	validation_0-logloss:0.45516
[714]	validation_0-logloss:0.45510
[715]	validation_0-logloss:0.45509
[716]	validation_0-logloss:0.45507
[717]	validation_0-logloss:0.45523
[718]	validation_0-logloss:0.45527
[719]	validation_0-logloss:0.45560
[720]	validation_0-logloss:0.45566
[721]	validation_0-logloss:0.45578
[722]	validation_0-logloss:0.45561
[723]	validation_0-logloss:0.45574
[724]	validation_0-logloss:0.45610
[725]	validation_0-logloss:0.45615
[726]	validation_0-logloss:0.45622
[727]	validation_0-logloss:0.45664
[728]	validation_0-logloss:0.45680
[729]	validation_0-logloss:0.45693
[730]	validation_0-logloss:0.45734
[731]	validation_0-logloss:0.45725
[732]	validation_0-logloss:0.45736
[733]	validation_0-logloss:0.45748
[734]	validation_0-logloss:0.45753
[735]	validation_0-logloss:0.45757
[736]	validation_0-logloss:0.45788
[737]	validation_0-l

[944]	validation_0-logloss:0.47112
[945]	validation_0-logloss:0.47140
[946]	validation_0-logloss:0.47139
[947]	validation_0-logloss:0.47144
[948]	validation_0-logloss:0.47163
[949]	validation_0-logloss:0.47166
[950]	validation_0-logloss:0.47185
[951]	validation_0-logloss:0.47193
[952]	validation_0-logloss:0.47203
[953]	validation_0-logloss:0.47206
[954]	validation_0-logloss:0.47215
[955]	validation_0-logloss:0.47240
[956]	validation_0-logloss:0.47261
[957]	validation_0-logloss:0.47267
[958]	validation_0-logloss:0.47269
[959]	validation_0-logloss:0.47272
[960]	validation_0-logloss:0.47276
[961]	validation_0-logloss:0.47275
[962]	validation_0-logloss:0.47277
[963]	validation_0-logloss:0.47282
[964]	validation_0-logloss:0.47293
[965]	validation_0-logloss:0.47308
[966]	validation_0-logloss:0.47307
[967]	validation_0-logloss:0.47320
[968]	validation_0-logloss:0.47326
[969]	validation_0-logloss:0.47328
[970]	validation_0-logloss:0.47339
[971]	validation_0-logloss:0.47350
[972]	validation_0-l